<a href="https://colab.research.google.com/github/MudassirABBASSi/Paper-pattern-detection/blob/main/paper_Pattern_Detection.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install 'git+https://github.com/facebookresearch/detectron2.git'

  Cloning https://github.com/facebookresearch/detectron2.git to /tmp/pip-req-build-7mq8mruf
  Running command git clone --filter=blob:none --quiet https://github.com/facebookresearch/detectron2.git /tmp/pip-req-build-7mq8mruf
  Resolved https://github.com/facebookresearch/detectron2.git to commit 9604f5995cc628619f0e4fd913453b4d7d61db3f
  Preparing metadata (setup.py) ... done


In [ ]:
# -*- coding: utf-8 -*-
"""paper Pattern Detection.ipynb

Automatically generated by Colab.

Original file is located at
    https://colab.research.google.com/drive/1IahUUFdkuKTmkozSoUlS05NeWEsJ2jMj
"""

# !pip install 'git+https://github.com/facebookresearch/detectron2.git'

"""Evalution Check

#10000 iteartion
"""

import os
import zipfile

# =================== Step 1: Extract the Dataset ===================
zip_path = "/content/CV Answear sheet.v1i.coco.zip"
extract_path = "/content/CV Answear sheet.v1i.coco"

if not os.path.exists(extract_path):
    with zipfile.ZipFile(zip_path, 'r') as zip_ref:
        zip_ref.extractall("/content/OMR_Answear_sheet")
    print("Dataset extracted successfully.")
else:
    print("Dataset already extracted.")

# =================== Step 2: Register the Dataset ===================
from detectron2.data.datasets import register_coco_instances
from detectron2.data import DatasetCatalog, MetadataCatalog

# Define dataset paths (update if necessary)
train_images_path = os.path.join(extract_path, "/content/OMR_Answear_sheet/CV Answear sheet.v1i.coco/train")
train_annotations_path = os.path.join(train_images_path, "/content/OMR_Answear_sheet/CV Answear sheet.v1i.coco/train/_annotations.coco.json")
valid_images_path = os.path.join(extract_path, "/content/OMR_Answear_sheet/CV Answear sheet.v1i.coco/valid")
valid_annotations_path = os.path.join(valid_images_path, "/content/OMR_Answear_sheet/CV Answear sheet.v1i.coco/valid/_annotations.coco.json")

# Verify that the paths exist:
assert os.path.exists(train_images_path), f"Train images not found at {train_images_path}"
assert os.path.exists(train_annotations_path), f"Train annotations not found at {train_annotations_path}"
assert os.path.exists(valid_images_path), f"Validation images not found at {valid_images_path}"
assert os.path.exists(valid_annotations_path), f"Validation annotations not found at {valid_annotations_path}"

# Dataset names
train_dataset_name = "cv_train"
valid_dataset_name = "cv_valid"

# Unregister if already registered
if train_dataset_name in DatasetCatalog.list():
    DatasetCatalog.remove(train_dataset_name)
    MetadataCatalog.remove(train_dataset_name)
if valid_dataset_name in DatasetCatalog.list():
    DatasetCatalog.remove(valid_dataset_name)
    MetadataCatalog.remove(valid_dataset_name)

register_coco_instances(train_dataset_name, {}, train_annotations_path, train_images_path)
register_coco_instances(valid_dataset_name, {}, valid_annotations_path, valid_images_path)
print("✅ Training and validation datasets registered successfully.")
print("Registered datasets:", DatasetCatalog.list())

# =================== Step 3: Visualize a Sample (Optional) ===================
import cv2
from detectron2.utils.visualizer import Visualizer

dataset_dicts = DatasetCatalog.get(train_dataset_name)
metadata = MetadataCatalog.get(train_dataset_name)
sample_data = dataset_dicts[0]
sample_image_path = sample_data["file_name"]
sample_image = cv2.imread(sample_image_path)
visualizer = Visualizer(sample_image[:, :, ::-1], metadata=metadata, scale=1.0)
output_vis = visualizer.draw_dataset_dict(sample_data)
cv2.imwrite("sample_image.jpg", output_vis.get_image()[:, :, ::-1])
print("✅ Sample visualization saved as 'sample_image.jpg'.")

# =================== Step 4: Configure the Model ===================
from detectron2.config import get_cfg
from detectron2 import model_zoo
from detectron2.engine import DefaultTrainer, DefaultPredictor

# Enable CUDA blocking for debugging (optional)
os.environ["CUDA_LAUNCH_BLOCKING"] = "1"

cfg = get_cfg()
cfg.merge_from_file(model_zoo.get_config_file("COCO-Detection/faster_rcnn_R_50_FPN_3x.yaml"))
cfg.DATASETS.TRAIN = (train_dataset_name,)
cfg.DATASETS.TEST = (valid_dataset_name,)
cfg.DATALOADER.NUM_WORKERS = 2
cfg.MODEL.WEIGHTS = model_zoo.get_checkpoint_url("COCO-Detection/faster_rcnn_R_50_FPN_3x.yaml")
cfg.SOLVER.IMS_PER_BATCH = 2
cfg.SOLVER.BASE_LR = 0.00025
# Increase iterations to 10,000 for better learning.
cfg.SOLVER.MAX_ITER =5000
cfg.MODEL.ROI_HEADS.BATCH_SIZE_PER_IMAGE = 128

# IMPORTANT: Set the number of classes to match your dataset.
cfg.MODEL.ROI_HEADS.NUM_CLASSES = 45

# (Optional) If you don't have a GPU, uncomment the next line:
# cfg.MODEL.DEVICE = "cpu"

print("✅ Model configuration completed.")

# =================== Step 5: Train the Model ===================
os.makedirs(cfg.OUTPUT_DIR, exist_ok=True)
trainer = DefaultTrainer(cfg)
trainer.resume_or_load(resume=False)
trainer.train()

# =================== Step 6: Inference on a Validation Image ===================
cfg.MODEL.WEIGHTS = os.path.join(cfg.OUTPUT_DIR, "model_final.pth")
cfg.MODEL.ROI_HEADS.SCORE_THRESH_TEST = 0.5
predictor = DefaultPredictor(cfg)

valid_images_list = os.listdir(valid_images_path)
if len(valid_images_list) == 0:
    raise ValueError("No validation images found!")
test_image_path = os.path.join(valid_images_path, valid_images_list[0])
test_image = cv2.imread(test_image_path)
outputs = predictor(test_image)

v = Visualizer(test_image[:, :, ::-1], metadata=MetadataCatalog.get(valid_dataset_name), scale=1.0)
result_vis = v.draw_instance_predictions(outputs["instances"].to("cpu"))
cv2.imwrite("inference_result.jpg", result_vis.get_image()[:, :, ::-1])
print("✅ Inference result saved as 'inference_result.jpg'.")



Dataset extracted successfully.
✅ Training and validation datasets registered successfully.
Registered datasets: ['coco_2014_train', 'coco_2014_val', 'coco_2014_minival', 'coco_2014_valminusminival', 'coco_2017_train', 'coco_2017_val', 'coco_2017_test', 'coco_2017_test-dev', 'coco_2017_val_100', 'keypoints_coco_2014_train', 'keypoints_coco_2014_val', 'keypoints_coco_2014_minival', 'keypoints_coco_2014_valminusminival', 'keypoints_coco_2017_train', 'keypoints_coco_2017_val', 'keypoints_coco_2017_val_100', 'coco_2017_train_panoptic_separated', 'coco_2017_train_panoptic_stuffonly', 'coco_2017_train_panoptic', 'coco_2017_val_panoptic_separated', 'coco_2017_val_panoptic_stuffonly', 'coco_2017_val_panoptic', 'coco_2017_val_100_panoptic_separated', 'coco_2017_val_100_panoptic_stuffonly', 'coco_2017_val_100_panoptic', 'lvis_v1_train', 'lvis_v1_val', 'lvis_v1_test_dev', 'lvis_v1_test_challenge', 'lvis_v0.5_train', 'lvis_v0.5_val', 'lvis_v0.5_val_rand_100', 'lvis_v0.5_test', 'lvis_v0.5_train_coc

roi_heads.box_predictor.bbox_pred.{bias, weight}
roi_heads.box_predictor.cls_score.{bias, weight}


[03/21 20:56:15 d2.engine.train_loop]: Starting training from iteration 0
[03/21 20:56:21 d2.utils.events]:  eta: 0:25:45  iter: 19  total_loss: 9.166  loss_cls: 3.803  loss_box_reg: 0.0542  loss_rpn_cls: 4.785  loss_rpn_loc: 0.4284    time: 0.3097  last_time: 0.3484  data_time: 0.0254  last_data_time: 0.0054   lr: 4.9953e-06  max_mem: 2174M
[03/21 20:56:28 d2.utils.events]:  eta: 0:26:35  iter: 39  total_loss: 6.626  loss_cls: 3.597  loss_box_reg: 0.311  loss_rpn_cls: 2.258  loss_rpn_loc: 0.4339    time: 0.3167  last_time: 0.3462  data_time: 0.0133  last_data_time: 0.0051   lr: 9.9902e-06  max_mem: 2182M
[03/21 20:56:34 d2.utils.events]:  eta: 0:25:59  iter: 59  total_loss: 4.693  loss_cls: 3.323  loss_box_reg: 0.363  loss_rpn_cls: 0.6171  loss_rpn_loc: 0.3571    time: 0.3137  last_time: 0.2991  data_time: 0.0076  last_data_time: 0.0055   lr: 1.4985e-05  max_mem: 2182M
[03/21 20:56:40 d2.utils.events]:  eta: 0:25:53  iter: 79  total_loss: 3.827  loss_cls: 2.614  loss_box_reg: 0.5419  

In [ ]:
import locale
locale.getpreferredencoding = lambda: 'UTF-8'
!pip install gradio
!pip install torch torchvision

In [ ]:
import torch
import pickle
import numpy as np
import cv2
from detectron2.utils.visualizer import Visualizer
from detectron2.data import MetadataCatalog

# Set device
DEVICE = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"✅ Using device: {DEVICE}")

# Load the model from the pickle file
def load_model(model_path):
    try:
        with open(model_path, "rb") as f:
            predictor = pickle.load(f)

        # Ensure model is on the correct device
        predictor.model.to(DEVICE)

        return predictor
    except Exception as e:
        print(f"❌ Error loading model: {e}")
        return None

# Load metadata
def setup_metadata(num_classes=80):
    """Set up dataset metadata for visualization"""
    dataset_name = "paper_detection"

    if dataset_name in MetadataCatalog.list():
        MetadataCatalog.remove(dataset_name)

    MetadataCatalog.get(dataset_name).set(thing_classes=[f"class_{i}" for i in range(num_classes)])

    return MetadataCatalog.get(dataset_name)

# Detect objects in an image
def detect_patterns(input_image):
    if input_image is None:
        return np.zeros((400, 600, 3), dtype=np.uint8), "❌ No image provided. Please upload an image."

    try:
        predictor = load_model("/content/model.pkl")
        if predictor is None:
            return np.zeros((400, 600, 3), dtype=np.uint8), "❌ Model failed to load."

        metadata = setup_metadata()

        # Convert image to OpenCV format
        img_array = np.array(input_image)
        img_array = cv2.cvtColor(img_array, cv2.COLOR_RGB2BGR)

        # Ensure image is moved to correct device
        img_tensor = torch.tensor(img_array, dtype=torch.float32).permute(2, 0, 1).unsqueeze(0).to(DEVICE)

        # Run inference
        with torch.no_grad():
            outputs = predictor(img_tensor)

        # Visualize results
        v = Visualizer(img_array, metadata=metadata, scale=1.0)
        result = v.draw_instance_predictions(outputs["instances"].to("cpu"))
        result_image = result.get_image()

        return result_image, "✅ Detection completed!"

    except Exception as e:
        return np.zeros((400, 600, 3), dtype=np.uint8), f"❌ Error: {e}"



✅ Using device: cuda


In [ ]:
import torch
import torch.nn as nn
import pickle

# Define a simple model (Replace with your actual model)
class SimpleModel(nn.Module):
    def __init__(self):
        super(SimpleModel, self).__init__()
        self.fc = nn.Linear(10, 2)  # Example: 10 input features → 2 output classes

    def forward(self, x):
        return self.fc(x)

# Initialize model
model = SimpleModel()

# Save the model using pickle
model_path = "model.pkl"
with open(model_path, "wb") as f:
    pickle.dump(model, f)

print(f"✅ Model saved as {model_path}")


✅ Model saved as model.pkl


#Frontend

# origial

In [ ]:
from detectron2 import model_zoo
cfg.merge_from_file(model_zoo.get_config_file("COCO-Detection/faster_rcnn_R_50_FPN_3x.yaml"))

import gradio as gr
import cv2
import torch
import pandas as pd
import numpy as np
from detectron2.engine import DefaultPredictor
from detectron2.config import get_cfg
from detectron2 import model_zoo
from detectron2.utils.visualizer import Visualizer
from detectron2.data import MetadataCatalog

# =================== Load the Trained Model ===================
cfg = get_cfg()
cfg.merge_from_file(model_zoo.get_config_file("COCO-Detection/faster_rcnn_R_50_FPN_3x.yaml"))
cfg.MODEL.WEIGHTS = "/content/output/model_final.pth"  # Update with your trained model path
cfg.MODEL.ROI_HEADS.SCORE_THRESH_TEST = 0.5
cfg.MODEL.ROI_HEADS.NUM_CLASSES = 45  # Ensure this matches your dataset
predictor = DefaultPredictor(cfg)

# Retrieve metadata (Ensure correct dataset is registered)
dataset_name = "cv_valid"  # Update with your actual dataset name
metadata = MetadataCatalog.get(dataset_name)

# =================== Function to Process Image ===================
def detect_objects(image):
    """
    Detect objects in an uploaded image, save annotated image and CSV results.
    """
    im = cv2.cvtColor(image, cv2.COLOR_RGB2BGR)  # Convert image from RGB to BGR
    outputs = predictor(im)
    instances = outputs["instances"].to("cpu")

    # =================== Count Detected Classes ===================
    class_counts = {}
    detected_classes = instances.pred_classes.numpy()

    if hasattr(metadata, "thing_classes") and metadata.thing_classes:
        class_names = metadata.thing_classes
    else:
        class_names = [f"Class_{i}" for i in range(cfg.MODEL.ROI_HEADS.NUM_CLASSES)]

    for cls in detected_classes:
        if cls < len(class_names):  # Ensure index is within bounds
            class_name = class_names[cls]
            class_counts[class_name] = class_counts.get(class_name, 0) + 1

    # Convert class counts to DataFrame and Save CSV
    df = pd.DataFrame(class_counts.items(), columns=["Class", "Count"])
    csv_file = "/content/detection_results.csv"
    df.to_csv(csv_file, index=False)

    # =================== Annotate Image ===================
    visualizer = Visualizer(im[:, :, ::-1], metadata=metadata, scale=1.0)
    out = visualizer.draw_instance_predictions(instances)
    annotated_image = out.get_image()[:, :, ::-1]

    return annotated_image, csv_file  # Return both annotated image and CSV file

# =================== Gradio Interface ===================
iface = gr.Interface(
    fn=detect_objects,
    inputs=gr.Image(type="numpy"),
    outputs=[gr.Image(type="numpy", label="Annotated Image"), gr.File(label="Download CSV Results")],
    title="Object Detection & CSV Export",
    description="Upload an image, detect objects, and download detection results as CSV."
)

iface.launch()


[03/21 21:22:58 d2.checkpoint.detection_checkpoint]: [DetectionCheckpointer] Loading from /content/output/model_final.pth ...
Running Gradio in a Colab notebook requires sharing enabled. Automatically setting `share=True` (you can turn this off by setting `share=False` in `launch()` explicitly).

Colab notebook detected. To show errors in colab notebook, set debug=True in launch()
* Running on public URL: https://7263f7df428e172853.gradio.live

This share link expires in 72 hours. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)


#url

In [ ]:
import os
import cv2
import pandas as pd
from detectron2.engine import DefaultPredictor
from detectron2.utils.visualizer import Visualizer
from detectron2.data import MetadataCatalog
from detectron2.config import get_cfg
from detectron2 import model_zoo

# =================== Load the Trained Model ===================
cfg = get_cfg()
cfg.merge_from_file(model_zoo.get_config_file("COCO-Detection/faster_rcnn_R_50_FPN_3x.yaml"))  # ✅ FIXED!
cfg.MODEL.ROI_HEADS.NUM_CLASSES = 45  # Ensure this matches your dataset
cfg.MODEL.WEIGHTS = "/content/output/model_final.pth"  # Load trained weights
cfg.MODEL.ROI_HEADS.SCORE_THRESH_TEST = 0.5  # Set confidence threshold

predictor = DefaultPredictor(cfg)  # Load the trained model

# =================== Function to Process an Image ===================
def process_image(image_path, output_csv="detection_results.csv"):
    """
    Detect objects in an image, visualize detections, and save results in CSV.
    """
    if not os.path.exists(image_path):
        print("❌ Error: Image file not found. Please enter a valid path.")
        return None, None

    # Read the input image
    image = cv2.imread(image_path)

    # Perform object detection
    outputs = predictor(image)

    # Get detected classes
    class_ids = outputs["instances"].pred_classes.cpu().numpy()

    # Load class names
    dataset_name = "cv_valid"  # Update with your dataset name
    metadata = MetadataCatalog.get(dataset_name)

    if hasattr(metadata, "thing_classes") and metadata.thing_classes:
        class_names = metadata.thing_classes
    else:
        class_names = [f"Unknown_Class_{i}" for i in range(45)]  # Handle missing class names

    # Convert class IDs to class names
    detected_classes = [class_names[i] if i < len(class_names) else f"Unknown_Class_{i}" for i in class_ids]

    # Count occurrences of each detected class
    class_counts = pd.Series(detected_classes).value_counts().reset_index()
    class_counts.columns = ["Class Name", "Count"]

    # Save results to a CSV file
    class_counts.to_csv(output_csv, index=False)

    print(f"✅ Object detection complete. Results saved in {output_csv}.")

    # =================== Visualize and Save the Detection Image ===================
    visualizer = Visualizer(image[:, :, ::-1], metadata=metadata, scale=1.0)
    out = visualizer.draw_instance_predictions(outputs["instances"].to("cpu"))
    result_image_path = "detection_result.jpg"
    cv2.imwrite(result_image_path, out.get_image()[:, :, ::-1])

    print(f"✅ Detection image saved as {result_image_path}.")

    return result_image_path, output_csv


# =================== Ask for an Image and Process It ===================
if __name__ == "__main__":
    image_path = input("📂 Enter the path to the image file: ")  # Ask user to enter image path
    output_image, csv_file = process_image(image_path)

    if output_image and csv_file:
        print(f"✅ Processed image saved at: {output_image}")
        print(f"✅ CSV file saved at: {csv_file}")


📂 Enter the path to the image file: /content/2.jpg
✅ Object detection complete. Results saved in detection_results.csv.
✅ Detection image saved as detection_result.jpg.
✅ Processed image saved at: detection_result.jpg
✅ CSV file saved at: detection_results.csv


#image uploade

In [ ]:
import os
import cv2
import pandas as pd
import ipywidgets as widgets
from IPython.display import display
from detectron2.engine import DefaultPredictor
from detectron2.utils.visualizer import Visualizer
from detectron2.data import MetadataCatalog
from detectron2.config import get_cfg
from detectron2 import model_zoo
from google.colab import files  # Required for image upload in Colab

# =================== Load the Trained Model ===================
cfg = get_cfg()
cfg.merge_from_file(model_zoo.get_config_file("COCO-Detection/faster_rcnn_R_50_FPN_3x.yaml"))
cfg.MODEL.ROI_HEADS.NUM_CLASSES = 10  # Update based on your dataset
cfg.MODEL.WEIGHTS = "/content/output/model_final.pth"  # Load trained weights
cfg.MODEL.ROI_HEADS.SCORE_THRESH_TEST = 0.5  # Set confidence threshold

predictor = DefaultPredictor(cfg)  # Load the trained model

# =================== Custom Mapping ===================
custom_labels = {
    "Class_0": "Serial No",
    "Class_1": "Month 4",
    "Class_2": "Week 3",
    "Class_3": "Class 11",
    "Class_4": "Section A",
    "Class_5": "B1-S",
    "Class_6": "B2-N",
    "Class_7": "B3-N",
    "Class_8": "B4-F",
    "Class_9": "Other",
}

# =================== Function to Process an Uploaded Image ===================
def process_uploaded_image(uploaded_filename, uploaded_file, output_csv="detection_results.csv"):
    """
    Detect objects in an uploaded image, visualize detections, and save results in CSV.
    """
    # Save uploaded image
    image_path = f"/content/{uploaded_filename}"
    with open(image_path, "wb") as f:
        f.write(uploaded_file)

    # Read the image
    image = cv2.imread(image_path)

    # Perform object detection
    outputs = predictor(image)

    # Get detected classes
    class_ids = outputs["instances"].pred_classes.cpu().numpy()

    # Convert class IDs to custom names and count occurrences
    class_counts = {name: 0 for name in custom_labels.values()}  # Initialize all counts to 0
    for class_id in class_ids:
        class_name = f"Class_{class_id}"
        if class_name in custom_labels:
            class_counts[custom_labels[class_name]] += 1

    # Convert to DataFrame
    df = pd.DataFrame(list(class_counts.items()), columns=["Class Name", "Count"])
    df.to_csv(output_csv, index=False)

    # ✅ PRINT THE CLASS COUNTS IN OUTPUT
    print("\n📊 **Detection Results:**\n")
    for class_name, count in class_counts.items():
        print(f"🔹 {class_name}: {count}")

    print(f"\n✅ Object detection complete. Results saved in {output_csv}.")

    # =================== Visualize and Save the Detection Image ===================
    dataset_name = "cv_valid"
    metadata = MetadataCatalog.get(dataset_name)
    visualizer = Visualizer(image[:, :, ::-1], metadata=metadata, scale=1.0)
    out = visualizer.draw_instance_predictions(outputs["instances"].to("cpu"))
    result_image_path = "/content/detection_result.jpg"
    cv2.imwrite(result_image_path, out.get_image()[:, :, ::-1])

    print(f"✅ Detection image saved as {result_image_path}.")

    return result_image_path, output_csv


# =================== Upload and Process Image ===================
def upload_and_process():
    """
    Function to upload an image in Jupyter/Colab and process it.
    """
    uploaded = files.upload()  # Open file upload dialog
    for filename, file_data in uploaded.items():  # Loop in case of multiple files
        output_image, csv_file = process_uploaded_image(filename, file_data)
        print(f"✅ Processed image saved at: {output_image}")
        print(f"✅ CSV file saved at: {csv_file}")

# Create a button to trigger the upload function
upload_button = widgets.Button(description="📂 Upload Image & Process")
upload_button.on_click(lambda x: upload_and_process())

# Display button in Jupyter Notebook or Google Colab
display(upload_button)


roi_heads.box_predictor.bbox_pred.{bias, weight}
roi_heads.box_predictor.cls_score.{bias, weight}


Button(description='📂 Upload Image & Process', style=ButtonStyle())

Saving 2.jpg to 2 (6).jpg

📊 **Detection Results:**

🔹 Serial No: 0
🔹 Month 4: 0
🔹 Week 3: 0
🔹 Class 11: 0
🔹 Section A: 0
🔹 B1-S: 0
🔹 B2-N: 0
🔹 B3-N: 0
🔹 B4-F: 0
🔹 Other: 0

✅ Object detection complete. Results saved in detection_results.csv.
✅ Detection image saved as /content/detection_result.jpg.
✅ Processed image saved at: /content/detection_result.jpg
✅ CSV file saved at: detection_results.csv
